In [47]:
import nibabel as nib
#from monai.transforms import Compose, LoadImaged, AddChanneld, ToTensord
from monai.inferers import sliding_window_inference
from monai.networks.nets import UNet
from monai.data import Dataset, DataLoader
import torch
import nibabel as nib
import numpy as np
import pandas as pd
import os
import glob

In [48]:

combined_roi_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
        'Left-Hippocampus',
        'Left-Amygdala',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
        'Right-Hippocampus',
        'Right-Amygdala',
    },
    'Left-Hippocampus':{
        'Left-Hippocampus'
    },

    'Right-Hippocampus': {
        'Right-Hippocampus'

    },

    'Left-Amygdala':{
        'Left-Amygdala'

    },

    'Right-Amygdala':{
        'Right-Amygdala'

    },

    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    },
    'Left-Total-Lateral-Ventricle': {
        'Left-Lateral-Ventricle',
        'Left-Inf-Lat-Vent'
    },
    'Right-Total-Lateral-Ventricle': {
        'Right-Lateral-Ventricle',
        'Right-Inf-Lat-Vent'
    },
    'Left-Lateral-Ventricle': {
        'Left-Lateral-Ventricle'
      
    },
    'Right-Lateral-Ventricle': {
        'Right-Lateral-Ventricle'
    },
     'Left-Inf-Lat-Vent':{
          'Left-Inf-Lat-Vent'
    },
    'Right-Inf-Lat-Vent': {
         'Right-Inf-Lat-Vent'

    }
    
}

In [14]:
def load_pickle_file(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data
# 파일열어보기 , PICKLE로 되어있는것

In [15]:

base_dir = '/mnt/drwelly1/Team_Planning/Brain_Volumetry/OASIS1_average_before'


def calculate_volume(nii_path):
    img = nib.load(nii_path) 
    data = img.get_fdata()
    header = img.header
    voxel_volume = np.prod(header.get_zooms())
    unique_elements, counts_elements = np.unique(data, return_counts=True)
    volumes = counts_elements * voxel_volume
    return dict(zip(unique_elements.astype(int), volumes))

In [16]:
import os
import pandas as pd



volume_data = []

for subject_dir in os.listdir(base_dir):
    full_subject_dir = os.path.join(base_dir, subject_dir)
    if not os.path.isdir(full_subject_dir) or 'FS222' not in subject_dir:
        continue
    
    double_subject_dir = os.path.join(full_subject_dir, subject_dir)
    if not os.path.isdir(double_subject_dir):
        continue
    
    mri_dir = os.path.join(double_subject_dir, 'mri')
    nii_path = os.path.join(mri_dir, 'aparc.DKTatlas+aseg.deep.nii.gz')
    
    if os.path.isfile(nii_path):
        try:
            vol_data = calculate_volume(nii_path)
            vol_data["subject"] = subject_dir
            volume_data.append(vol_data)
        except OSError as e:
            print(f"Error processing file {nii_path}, skipped  Error: {e}")
    else:
        print(f"File not found, skipped: {nii_path}")

volume_df = pd.DataFrame(volume_data)

In [17]:
volume_df

,0,2,4,5,7,8,10,11,12,13,...,2025,2026,2027,2028,2029,2030,2031,2034,2035,subject
0,1.585129e+07,171572.713376,32362.392505,1800.406972,9353.266857,40372.814383,5682.137913,3467.729947,3853.947941,1568.195976,...,7394.050887,1350.733979,8294.082873,22546.761655,8465.239871,11862.654819,7333.339888,815.310988,4970.069924,OAS1_0299_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
1,1.554823e+07,244994.606254,6098.882907,226.036997,14184.078783,60595.408073,8374.001872,4013.442939,6197.323905,2308.732965,...,11765.585820,3042.066953,13919.625787,30660.769531,11330.318827,16043.824755,9494.582855,1104.459983,6477.897901,OAS1_0131_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
2,1.596080e+07,163840.121495,5433.119917,254.505996,9949.743848,42022.301357,5755.882912,2730.622958,3822.734942,1389.149979,...,6557.473900,1946.867970,9097.731861,21019.725679,8214.506874,12115.102815,6533.463900,835.204987,4663.427929,OAS1_0355_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
3,1.589780e+07,171497.253377,4273.436935,132.054998,10937.240833,45108.615310,6493.675901,2936.765955,4212.725936,1671.438974,...,9312.106858,1866.605971,9506.930855,21304.415674,9662.652852,13207.557798,8855.230865,882.538987,4611.977929,OAS1_0387_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
4,1.591131e+07,173067.507353,31278.169522,1251.606981,10073.909846,36837.856437,6042.287908,3021.829954,5478.738916,1675.211974,...,6732.060897,2011.008969,5719.524913,16861.879742,6858.627895,10338.705842,6762.587897,478.141993,5180.328921,OAS1_0380_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,1.570715e+07,213786.065731,6125.293906,340.941995,11954.235817,50983.519220,7678.397883,3870.411941,4406.177933,1829.218972,...,10011.140847,2199.658966,13051.835800,24397.932627,10046.469846,14418.004780,9309.705858,765.918988,4897.353925,OAS1_0104_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
432,1.540799e+07,276191.484776,8427.852871,291.206996,15734.781759,70181.228927,9796.765850,4182.198936,5830.999911,2202.059966,...,12461.875809,2615.374960,14836.121773,37123.918432,13238.427798,17111.240738,11983.733817,1225.195981,6481.670901,OAS1_0108_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
433,1.577957e+07,196902.576989,26935.103588,1242.688981,11854.765819,48286.167262,6630.532899,3762.023942,4206.894936,1773.652973,...,8660.406868,1633.708975,8134.244876,22284.366659,8212.791874,12101.725815,7510.670885,865.388987,5632.402914,OAS1_0425_MR1_Tag_pr100_sr100_ppf100_spf100_ip...
434,1.587457e+07,183037.145201,29520.637549,486.716993,11075.469831,41284.851369,5793.955911,2998.848954,3614.533945,1580.543976,...,7597.449884,2158.155967,7211.917890,19163.752707,8733.465866,10600.757838,7614.599884,791.986988,5520.927916,OAS1_0035_MR1_Tag_pr100_sr100_ppf100_spf100_ip...


In [21]:
volume_df['subject'] = volume_df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf143_FS222', '')


In [22]:
#volume_df['subject']

0      OAS1_0299_MR1
1      OAS1_0131_MR1
2      OAS1_0355_MR1
3      OAS1_0387_MR1
4      OAS1_0380_MR1
           ...      
431    OAS1_0104_MR1
432    OAS1_0108_MR1
433    OAS1_0425_MR1
434    OAS1_0035_MR1
435    OAS1_0315_MR1
Name: subject, Length: 436, dtype: object

In [24]:
excel_path = '/mnt/drwelly1/SeoyoungLim/Oasis_analysis/DrWelly_Volumetry_Colorcode.xlsx'

In [37]:
def load_excel_as_lut(excel_path):
    # Load the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_path)
    
    # Create a dictionary to hold your LUT
    lut = {}
    
    # Iterate over the rows of the DataFrame and populate the dictionary
    for index, row in df.iterrows():
        # Assuming the ID is in the first column and the label in the second
        lut[row[0]] = row[1]
    
    return lut

freesurfer_lut= load_excel_as_lut(excel_path)

/tmp/ipykernel_3705242/2193877719.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lut[row[0]] = row[1]


In [38]:
# 모든 컬럼 이름을 변경
new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in volume_df.columns]


volume_df.columns = new_columns

ValueError: invalid literal for int() with base 10: 'Unknown'

In [49]:
def update_df_columns_keep_subject(df, roi_dict):
    
    columns_to_remove = []
    
   
    protected_columns = ['subject','Left-Hippocampus','Right-Hippocampus', 'Left-Amygdala','Right-Amygdala', 'Left-Inf-Lat-Vent','Right-Inf-Lat-Vent'
                        ,'Left-Lateral-Ventricle','Right-Lateral-Ventricle']
                         

                         
    for roi, sub_columns in roi_dict.items():
      
        relevant_columns = [col for col in df.columns if col.split(' ')[0] in sub_columns and col not in protected_columns]
        
    
        if relevant_columns:
         
            df[roi] = df[relevant_columns].fillna(0).sum(axis=1)
          
            columns_to_remove.extend(relevant_columns)
 
    df = df.drop(columns=[col for col in columns_to_remove if col not in protected_columns], errors='ignore')
    
    
    return df

In [50]:


new_df1 = update_df_columns_keep_subject(volume_df.copy(), combined_roi_dict)

In [51]:

brain_areas = set()
left_right_columns = []

for col in new_df1.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_df1[area] = new_df1.get(left_col, 0) + new_df1.get(right_col, 0)

volume_df_final = new_df1.drop(columns=[col for col in left_right_columns if col not in ['subject']])

In [55]:
volume_df_final = volume_df_final.drop(columns = ['WM-hypointensities']) 

In [56]:
volume_df_final.columns

Index(['3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'CSF', 'subject',
       'Hippocampus', 'Accumbens-area', 'Lateral-Ventricle', 'Amygdala',
       'Cingulate', 'Frontal', 'VentralDC', 'Occipital', 'Pallidum',
       'Thalamus', 'Combined-Cerebral-WM', 'Temporal', 'Parietal', 'Caudate',
       'Cerebellum', 'Inf-Lat-Vent', 'Insula', 'Putamen'],
      dtype='object')

In [57]:
volume_df_final.to_csv('all_oasis1_average_before_swift_fastsurfer_20240430_FFinal.csv')

In [58]:
pwd

'/home/limseoyoung/debug'